In [10]:
import os
import glob
import pickle
from tqdm import tqdm
import numpy as np

from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model

print("Caricamento del modello InceptionV3...")
base_model = InceptionV3(weights='imagenet')

Caricamento del modello InceptionV3...


In [ ]:
model = Model(inputs=base_model.input, outputs=base_model.get_layer('avg_pool').output)
print("Modello per l'estrazione delle feature creato.")


Modello per l'estrazione delle feature creato.


In [ ]:
def extract_feature(img_path, model):
    """
    Carica un'immagine, la preprocessa per InceptionV3 e ne estrae le feature.
    """
    try:
        img = image.load_img(img_path, target_size=(299, 299))
        
        x = image.img_to_array(img)
        
        x = np.expand_dims(x, axis=0)
        
        x = preprocess_input(x)
        
        feature = model.predict(x, verbose=0)
        
        return feature.flatten() 

    except Exception as e:
        print(f"\nErrore durante l'elaborazione di {img_path}: {e}")
        return None


In [ ]:
images_path = r"..\data\Flickr8k_Dataset\Images" 
if not os.path.exists(images_path):
    raise FileNotFoundError(f"La cartella delle immagini non è stata trovata in: {images_path}")

In [ ]:

image_files = glob.glob(os.path.join(images_path, "*.jpg"))
print(f"Trovate {len(image_files)} immagini da processare.")

output_folder = "features"
os.makedirs(output_folder, exist_ok=True)

Trovate 8091 immagini da processare.


In [ ]:

features_dict = {}
for img_path in tqdm(image_files, desc="Estrazione feature"):
    image_id = os.path.basename(img_path).split('.')[0]
    feature_vector = extract_feature(img_path, model)
    
    if feature_vector is not None:
        features_dict[image_id] = feature_vector

Estrazione feature: 100%|██████████| 8091/8091 [46:56<00:00,  2.87it/s]  


In [ ]:

output_path = os.path.join(output_folder, "image_features.pkl")
with open(output_path, "wb") as f:
    pickle.dump(features_dict, f)

print(f"\n✔️ Estrazione completata. {len(features_dict)} feature salvate in '{output_path}'.")


✔️ Estrazione completata. 8091 feature salvate in 'features\image_features.pkl'.
